<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Good_faiss_search_wiki_semantic_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence-transformers
!pip install wikipedia
!pip install nltk
!pip install faiss-cpu

ERROR: You must give at least one requirement to install (see "pip help install")
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.4 MB/s eta 0:00:00


In [11]:
from sentence_transformers import SentenceTransformer, util
import wikipedia
import nltk
import faiss  # For efficient similarity search
import hashlib
import functools

# Download necessary NLTK data (if not already present)
nltk.download('punkt_tab')

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-mpnet-base-v2')  # or any other suitable model

# Cache for Wikipedia fetches and embeddings
@functools.lru_cache(maxsize=128)  # Adjust cache size as needed
def get_wikipedia_sentences(query):
    try:
        page = wikipedia.page(query)
        sentences = nltk.sent_tokenize(page.content)
        return sentences
    except wikipedia.exceptions.PageError:
        print(f"Error: Wikipedia page not found for '{query}'")
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Error: Disambiguation error for '{query}'. Options: {e.options}")
        return None

# Cache for sentence embeddings
@functools.lru_cache(maxsize=1024) # Adjust as needed
def get_embeddings(sentences):
    # Convert the list of sentences to a tuple before hashing
    sentences_tuple = tuple(sentences)
    # Now use sentences_tuple as the key for caching
    return model.encode(sentences_tuple)



def search_wikipedia(query, top_k=10):
    sentences = get_wikipedia_sentences(query)
    if sentences is None:
        return []

    query_embedding = model.encode([query])[0]
    # Pass the list of sentences directly to get_embeddings
    sentence_embeddings = model.encode(sentences)

    # FAISS for efficient similarity search
    dim = sentence_embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # Use FlatIP for cosine similarity
    index.add(sentence_embeddings)

    D, I = index.search(query_embedding.reshape(1, -1), top_k)  # Search for top_k
    results = [(sentences[i], score) for i, score in zip(I[0], D[0])]

    return results


# Example usage
query = "Amitabh Bachan films "
results = search_wikipedia(query)

if results:
    print(f"Top sentences related to '{query}':\n")
    for sentence, score in results:
        print(f"Score: {score:.4f}: {sentence}\n")



# Chunking Example (if needed for very long Wikipedia articles):
def chunk_text(text, chunk_size=500): # Adjust chunk_size as needed.
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        current_chunk.append(sentence)
        current_length += len(sentence)
        if current_length >= chunk_size:
            chunks.append(" ".join(current_chunk)) # Join sentences into a chunk
            current_chunk = []
            current_length = 0
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks


# Example chunking usage:
#  page = wikipedia.page("Artificial intelligence")
#  chunks = chunk_text(page.content)
#  # Then process chunks individually

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Top sentences related to 'Amitabh Bachan films ':

Score: 0.7120: Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema.

Score: 0.6943: (2009)
Ra.One (2011)
Kahaani (2012)
Krrish 3 (2013)
Mahabharat (2013)
Kochadaiiyaan (Hindi Version) (2014)
CBI documentary (2014) – sanctioned by Central Bureau of Investigation
The Ghazi Attack (2017)
Firangi (2017)


=== Business investments ===
Around 1994, Bachchan started Amitabh Bachchan Corporation Ltd (ABCL), an event management, production and distribution company.

Score: 0.6535: Bachchan attempted to revive his acting career, and eventually had commercial success with Bade Miyan Chote Miyan (1998) and Major Saab (1998), and received positive reviews for Sooryavansham (1999), but other films such as Lal Baadshah (1999) and Kohram (1999) were box office failures.

Score: 0.6350: This was the first movie in which Bachchan played a double role.

Score: 0.6184: